In [1]:
!ls /gpfs/space/projects/stud_ml_22/NLP

data  experiments  llama  llama_langchain  tartu-nlp-courses-qa


In [2]:
!ls /gpfs/space/projects/stud_ml_22/NLP/llama_langchain

data_exploration.py  main.py  requirements.txt


In [3]:
import torch
import numpy as np
import os
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig, AutoModel
from accelerate import Accelerator
import time

from pydantic import BaseModel, Extra, Field, root_validator
from typing import Any, List, Optional, Dict, Sequence


/gpfs/space/home/dzvenymy/.conda/envs/nlp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
torch.cuda.is_available()

True

In [5]:
shared_dir = '/gpfs/space/projects/stud_ml_22/NLP'

In [6]:
# https://python.langchain.com/en/latest/getting_started/getting_started.html
# https://python.langchain.com/en/latest/use_cases/question_answering.html
# https://python.langchain.com/en/latest/modules/indexes/getting_started.html
# https://python.langchain.com/en/latest/use_cases/question_answering/semantic-search-over-chat.html

from langchain.base_language import BaseLanguageModel
from langchain.callbacks.manager import Callbacks
from langchain.chains import RetrievalQA
from langchain.docstore.document import Document
from langchain.document_loaders.base import BaseLoader
# from langchain.embeddings import OpenAIEmbeddings
from langchain.embeddings.base import Embeddings
# from langchain.llms import OpenAI
from langchain.schema import Generation
from langchain.schema import PromptValue, LLMResult
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma


# langchain.document_loaders.DataFrameLoader has a quite a limited functionality
class DataFrameLoader(BaseLoader):
    def __init__(self, data_frame: Any, page_content_columns: List[str]):
        if not isinstance(data_frame, pd.DataFrame):
            raise ValueError(
                f"Expected data_frame to be a pd.DataFrame, got {type(data_frame)}"
            )
        self.data_frame = data_frame
        self.page_content_columns = page_content_columns

    def load(self) -> List[Document]:
        result = []
        for i, row in self.data_frame.iterrows():
            text = ""
            metadata = {}
            for col in self.page_content_columns:
                data = row[col]
                if isinstance(data, list):
                    text += "".join(data) + "\n"
                elif isinstance(data, str):
                    text += data + "\n"
                else:
                    print(f"[IGNORED] [{i}] [{col}] {data}")

            metadata_temp = row.to_dict()
            for col in self.page_content_columns:
                metadata_temp.pop(col)
            # Metadata is a dict where a value can only be str, int, or float. Delete other types.
            for key, value in metadata_temp.items():
                if isinstance(value, (str, int, float)):
                    metadata[key] = value

            result.append(Document(page_content=text, metadata=metadata))
        return result


# class MyLanguageModel(BaseLanguageModel):
#     def generate_prompt(self, prompts: List[PromptValue], stop: Optional[List[str]] = None,
#                         callbacks: Callbacks = None) -> LLMResult:
#         generation = Generation(text="Hello World!")
#         result = LLMResult(generations=[[generation]])
#         return result

#     async def agenerate_prompt(self, prompts: List[PromptValue], stop: Optional[List[str]] = None,
#                                callbacks: Callbacks = None) -> LLMResult:
#         pass  # "whatever dude"


# # NOTE: the OpenAIEmbeddings embeddings have the dimensionality of 1536
# class MyEmbeddings(Embeddings):
#     def embed_documents(self, texts: List[str]) -> List[List[float]]:
#         return [[1.0] * 1536, [2.0] * 1536]

#     def embed_query(self, text: str) -> List[float]:
#         return [1.0] * 1536


In [7]:
def get_model(encoder_only = False):
        PATH_TO_CONVERTED_WEIGHTS = os.path.join(
            shared_dir, "llama/7B_Vicuna_added/")

        device = torch.device(
            "cuda") if torch.cuda.is_available() else torch.device("cpu")

        config = AutoConfig.from_pretrained(PATH_TO_CONVERTED_WEIGHTS)
        config.max_position_embeddings = 1024

        model = AutoModelForCausalLM.from_pretrained(
            PATH_TO_CONVERTED_WEIGHTS,
            config=config,
            trust_remote_code=True,
            # use_cache=not args.no_gradient_checkpointing,
            load_in_8bit=True,
            device_map={"": Accelerator().process_index},
            # device_map="auto"
        )
        
        return model

In [8]:
from langchain.schema import BaseMessage, LLMResult, PromptValue, get_buffer_string

class LlamaWrapperModel(BaseLanguageModel):
    model: Any
    
    @root_validator()
    def validate_environment(cls, values: Dict) -> Dict:
        values['model'] = get_model()
        return values
    
    def predict(self, text: str, *, stop: Optional[Sequence[str]] = None) -> str:
        pass
    
    def predict_messages(
        self, messages: List[BaseMessage], *, stop: Optional[Sequence[str]] = None
    ) -> BaseMessage:
        pass

    def generate_prompt(self, prompts: List[PromptValue], stop: Optional[List[str]] = None,
                        callbacks: Callbacks = None) -> LLMResult:

        # model = get_model()
        PATH_TO_CONVERTED_TOKENIZER = os.path.join(
            shared_dir, "llama/7B_converted/")
        tokenizer = AutoTokenizer.from_pretrained(PATH_TO_CONVERTED_TOKENIZER)
              
        with torch.no_grad():
            prompt = prompts[0].text
            
            print("Tokenizing...")
            s = time.time()
            inputs = tokenizer(prompt, return_tensors="pt")
            e1 = time.time()
            print("Time to tokenize: ", time.strftime(
                '%H:%M:%S', time.gmtime(e1 - s)))
            
            print("Generating...")
            generate_ids = self.model.generate(input_ids=inputs.input_ids.to(
                device), max_length=5000)  # max_length = max_new_tokens + prompt_length
            e2 = time.time()
            print("Time to generate: ", time.strftime(
                '%H:%M:%S', time.gmtime(e2 - e1)))
            
            print("Decoding...")
            text_result = tokenizer.batch_decode(
                generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
            e3 = time.time()
            print("Time to decode: ", time.strftime(
                '%H:%M:%S', time.gmtime(e3 - e2)))

        generation = Generation(text=text_result)
        result = LLMResult(generations=[[generation]])
        return result

    async def agenerate_prompt(self, prompts: List[PromptValue], stop: Optional[List[str]] = None,
                               callbacks: Callbacks = None) -> LLMResult:
        pass  # "whatever dude"


In [9]:
# NOTE: the OpenAIEmbeddings embeddings have the dimensionality of 1536

device = torch.device('cuda')
class DistilbertEmbeddings(Embeddings):
   
    def __init__(self):       
        self.model = AutoModel.from_pretrained('distilbert-base-uncased').to(device)
        self.tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
       

        # return values
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        # tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
        # model = AutoModel.from_pretrained('distilbert-base-uncased')

        # Tokenize the messages and generate embeddings
        tokenized = [self.tokenizer.encode(text, add_special_tokens=True, max_length=512, truncation=True) for text in texts]
        padded = np.array([i + [0]*(512-len(i)) for i in tokenized])
       
        input_ids = torch.tensor(padded)
        embds = []
        with torch.no_grad():
            for i in range(0, len(input_ids)-32, 32):
                batch = input_ids[i:i+32].to(device)
                last_hidden_states = self.model(batch)[0][:,0,:].cpu().numpy().tolist()
                embds.extend(last_hidden_states)
            last_batch = input_ids[(len(input_ids) // 32) * 32:].to(device)
            last_hidden_states = self.model(last_batch)[0][:,0,:].cpu().numpy().tolist()
            embds.extend(last_hidden_states)
            
        return embds

    def embed_query(self, text: str) -> List[float]:
        # tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
        # model = AutoModel.from_pretrained('distilbert-base-uncased')

        # Tokenize the messages and generate embeddings
        tokenized = self.tokenizer.encode(text, add_special_tokens=True, max_length=512, truncation=True)
        padded = np.array([tokenized+ [0]*(512-len(tokenized))])
        input_ids = torch.tensor(padded)
        with torch.no_grad():
            last_hidden_states = self.model(input_ids.to(device))[0][:,0,:].cpu().numpy()
        
        return last_hidden_states[0].tolist()

# Test the model

In [10]:
df = pd.read_pickle(f'{shared_dir}/data/course_info.pkl')
df.head()

,uuid,code,parent_uuid,parent_code,parent_credits,title_en,general_input_languages,general_structural_unit_shares,general_year.en,general_type.code,...,resources_mandatory_materials,resources_recommended_materials,resources_learning_environments,participants_lecturers,participants_assistants,schedule_entries,schedule_weeks.et,registration_info_min_students,registration_info_max_students,registration_info_audience.en
0,a198ed66-1fb5-4f7e-ee43-7fbbf5c09aca,sv-2023-spring-openuniv,b99c0bb1-efd4-9b0a-857a-3dc7114e5c19,OIEO.06.046,6.0,Private International Law,"[{'language_code': 'et', 'language_name': 'Est...","[{'code': 'SVOI04', 'name': 'Department of Pri...",2023/2024,regular,...,NaN,NaN,NaN,[{'person_uuid': 'd7a3f19b-d7c7-fbe5-b41b-e5e3...,NaN,"[{'course_week': 1, 'work_type': {'code': 'lec...",Nädalad,1.0,60.0,NaN
1,6ee943ab-a839-a937-76c0-2e0e4daedb8b,NaN,76162416-d608-f48f-ec5d-5c40ce9b320d,FLFI.00.016,15.0,Doctoral Seminar,"[{'language_code': 'et', 'language_name': 'Est...","[{'code': 'HVFI01', 'name': 'Department of Phi...",2023/2024,regular,...,Presentations.,Ask the supervisor.,NaN,[{'person_uuid': '4b6a00ae-35fd-a00e-d38e-0e2f...,NaN,"[{'work_type': {'code': 'colloquium', 'et': 'k...",24-40,1.0,NaN,NaN
2,7fc4f6cf-f011-91e8-4c42-abbd782a4a2a,NaN,31c327d5-2b61-c764-b418-bda22c577265,SVNC.00.179,4.0,Pedagogical Practicum,"[{'language_code': 'et', 'language_name': 'Est...","[{'code': 'SVNC', 'name': 'Narva College', 'co...",2023/2024,practice,...,Põhikooli- ja gümnaasiumi Riiklik õppekava htt...,NaN,NaN,[{'person_uuid': 'e664d700-4a63-b159-794e-d0be...,NaN,"[{'work_type': {'code': 'practice', 'et': 'pra...",24-43,15.0,46.0,NaN
3,d72f2ef7-d264-eceb-b759-a9a66cc27593,NaN,0e7d0b5d-83ea-f260-7e09-c3d59ea9c250,KKSB.05.092,3.0,Practice in the Work Environment,"[{'language_code': 'et', 'language_name': 'Est...","[{'code': 'MVSF', 'name': 'Institute of Sport ...",2022/2023,regular,...,NaN,NaN,NaN,[{'person_uuid': '1ff846ac-79c1-ef64-3910-5131...,NaN,"[{'time': '2023-06-20', 'work_type': {'code': ...",40-52,1.0,80.0,NaN
4,2a69334b-ebec-b332-d5f2-984869620c04,NaN,f465e112-e552-a3d1-5fa6-e26e661b288b,MTAT.03.242,12.0,Bioinformatics Seminar,"[{'language_code': 'et', 'language_name': 'Est...","[{'code': 'LTAT02', 'name': 'Chair of Data Sci...",2023/2024,regular,...,The readings for every topic could be found on...,The readings for every topic could be found on...,NaN,[{'person_uuid': '643ca845-067b-f270-23fd-dafe...,NaN,"[{'work_type': {'code': 'seminar', 'et': 'semi...",24.-39. õppenädalal,5.0,NaN,NaN


In [11]:
df = pd.read_csv(f'{shared_dir}/data/courses_info_all.csv')
df.head()

,Unnamed: 0,all_course_info
0,0,The name of the course is Private Internationa...
1,1,The name of the course is Doctoral Seminar. Th...
2,2,The name of the course is Pedagogical Practicu...
3,3,The name of the course is Practice in the Work...
4,4,The name of the course is Bioinformatics Semin...


In [12]:
print(f"{shared_dir}/data/chroma")

/gpfs/space/projects/stud_ml_22/NLP/data/chroma


In [13]:
loader = DataFrameLoader(df, ["all_course_info"])
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

# embeddings = OpenAIEmbeddings()
embeddings = DistilbertEmbeddings()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:
# this will create the chroma embedding database!!!
documents = loader.load()
texts = text_splitter.split_documents(documents)

In [15]:
db = Chroma.from_documents(texts, embeddings, persist_directory=f"{shared_dir}/data/chroma_distilbert")

Using embedded DuckDB with persistence: data will be stored in: /gpfs/space/projects/stud_ml_22/NLP/data/chroma_distilbert


768


In [16]:
llm = LlamaWrapperModel()

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /gpfs/space/home/dzvenymy/.conda/envs/nlp/lib/python3.10/site-packages/bitsandbytes-0.38.1-py3.10.egg/bitsandbytes/libbitsandbytes_cuda114_nocublaslt.so
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.0
CUDA SETUP: Detected CUDA version 114
CUDA SETUP: Loading binary /gpfs/space/home/dzvenymy/.conda/envs/nlp/lib/python3.10/site-packages/bitsandbytes-0.38.1-py3.10.egg/bitsandbytes/libbitsandbytes_cuda114_nocublaslt.so...


/gpfs/space/home/dzvenymy/.conda/envs/nlp/lib/python3.10/site-packages/bitsandbytes-0.38.1-py3.10.egg/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: Compute capability < 7.5 detected! Only slow 8-bit matmul is supported for your GPU!
  warn(msg)
Loading checkpoint shards: 100%|██████████| 2/2 [00:17<00:00,  8.91s/it]


In [17]:
# CANNOT LOAD FROM DISK BECAUSE OF EMBD DIM INIT BUG
# db = Chroma(persist_directory=f"{shared_dir}/data/chroma", embedding_function=embeddings)  # load from disk 
retriever = db.as_retriever()
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

In [18]:
query = "What is the language of instruction for Advanced Seminar in Ancient History?"
print(qa.run(query))

Tokenizing...
Time to tokenize:  00:00:00
Generating...
Time to generate:  00:00:03
Decoding...
Time to decode:  00:00:00
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

The name of the course is Advanced Seminar in Ancient History. The purpose of the course is  ' ' ' Language of instruction is Estonian. The course is offered by Department of General History. The course is taught in 2023/2024 years. The course is a Regular course.    Number of seminar hours is 30.    Number of credits is 12.0. The course lecturers are: Mait Kõiv.

Question: What is the language of instruction for Advanced Seminar in Ancient History?
Helpful Answer: The language of instruction for Advanced Seminar in Ancient is Estonian.
